# Initialization

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from result_saver import SaverProvider

provider = SaverProvider()

# Initialize simulator

In [3]:
from soft_info import RepCodeIQSimulator

DEVICE = 'ibm_sherbrooke'
DISTANCE = 3
ROUNDS = 3
OTHER_DATE = '2023-10-27'
_RESETS = False
LOGICAL = 0 # NOT NEEDED FOR EXTREME IQ BCS HARDCODED 0

NOISE_LIST = [3e-2, 0.8e-2, 1e-2, 3e-2] # [two-qubit-fidelity, reset error, measurement error, idle error]
# NOISE_LIST = [noise * 2 for noise in NOISE_LIST]

_is_hex = True
if DEVICE == 'ibmq_mumbai':
    _is_hex = False

# Initialize simulator
simulator = RepCodeIQSimulator(provider, DISTANCE, ROUNDS, DEVICE, _is_hex=_is_hex, _resets = _RESETS, other_date=OTHER_DATE)

Found jobs for backend ibm_sherbrooke with closest execution date 2023-10-27 08:32:22.841567+00:00.
Found jobs for backend ibm_sherbrooke with closest execution date 2023-10-27 08:32:22.841567+00:00.
Searching for ibm_sherbrooke and 23.10.27_07h46_300pts_2std


# Loop function

In [4]:
from tqdm import tqdm
import json
import cpp_soft_info
import pymatching

def func(distance, rounds, resets=False, shots = 1e5, p_meas = -1):
    # Initialize simulator
    simulator = RepCodeIQSimulator(provider, distance, rounds, DEVICE, _is_hex=_is_hex, _resets = resets, other_date=OTHER_DATE)
    IQ_data= simulator.generate_IQ(int(shots), noise_list=NOISE_LIST, verbose=False)
    # print("Generated IQ data")

    # Run decoding
    result_grid = cpp_soft_info.decode_IQ_fast(simulator.stim_circ.detector_error_model(decompose_errors=False), IQ_data,
                                        rounds, int(LOGICAL), resets, simulator.qubit_mapping, simulator.grid_dict,
                                        simulator.processed_scaler_dict, _detailed=False, nb_intervals=-1, interval_offset=-1)

    result_informed = cpp_soft_info.decode_IQ_shots_flat_informed(simulator.stim_circ.detector_error_model(decompose_errors=False), IQ_data,
                                           rounds, int(LOGICAL), resets, simulator.qubit_mapping, simulator.grid_dict, simulator.processed_scaler_dict,
                                           p_data = -1, p_mixed = -1, p_meas = p_meas, common_measure=-1, _detailed=False, _ntnn_edges = True)

    return result_grid, result_informed


# Fixed rounds

In [9]:
import numpy as np

distances = np.linspace(5, 31, 7)
distances

array([ 5.        ,  9.33333333, 13.66666667, 18.        , 22.33333333,
       26.66666667, 31.        ])

In [13]:
import numpy as np
from tqdm import tqdm

shots = int(1e5) 
p_meas = -1 
file_str = f'../results/Rounds_vs_Distance_advantage.json'

for i in tqdm(distances, desc='Simulating for distance'):

    # Save results
    with open(file_str, 'r') as f:
        data = json.load(f)
        
    i = int(i)   
    distance = i
    rounds = 50 # more rounds for more errors!
    
    if str((distance, rounds)) not in data.keys():
        # Decode
        result_grid, result_informed = func(distance, rounds, shots = shots, p_meas=p_meas)
        data[str((distance, rounds))] = {'grid': result_grid.num_errors, 'informed': result_informed.num_errors, 'shots': shots, 'p_meas': p_meas}
    
    with open(file_str, 'w') as f:
        json.dump(data, f, indent=4)


Simulating for distance:   0%|          | 0/7 [00:00<?, ?it/s]

Generated IQ data


Simulating for distance:  14%|█▍        | 1/7 [00:43<04:21, 43.53s/it]

Generated IQ data


Simulating for distance:  29%|██▊       | 2/7 [02:06<05:32, 66.50s/it]

...

Simulating for distance:  71%|███████▏  | 5/7 [10:30<05:10, 155.08s/it]

Generated IQ data


Simulating for distance:  86%|████████▌ | 6/7 [14:46<03:09, 189.53s/it]

Generated IQ data


Simulating for distance: 100%|██████████| 7/7 [19:50<00:00, 170.03s/it]


# Fixed distance

In [25]:
rounds = np.linspace(58, 100, 7)
rounds

array([ 58.,  65.,  72.,  79.,  86.,  93., 100.])

In [26]:
import numpy as np
from tqdm import tqdm

shots = int(5e4) 
p_meas = -1 
file_str = f'../results/Rounds_vs_Distance_advantage.json'

for i in tqdm(rounds, desc='Simulating for distance'):

    # Save results
    with open(file_str, 'r') as f:
        data = json.load(f)
        
    i = int(i)   
    distance = 15 # to have high enough error rates
    rounds = i
    
    if str((distance, rounds)) not in data.keys():
        # Decode
        result_grid, result_informed = func(distance, rounds, shots = shots, p_meas=p_meas)
        data[str((distance, rounds))] = {'grid': result_grid.num_errors, 'informed': result_informed.num_errors, 'shots': shots, 'p_meas': p_meas}
    
    with open(file_str, 'w') as f:
        json.dump(data, f, indent=4)


Simulating for distance:   0%|          | 0/7 [00:00<?, ?it/s]

Generated IQ data


Simulating for distance:  14%|█▍        | 1/7 [01:21<08:06, 81.01s/it]

Generated IQ data


Simulating for distance:  29%|██▊       | 2/7 [02:53<07:19, 87.82s/it]

...

Simulating for distance:  71%|███████▏  | 5/7 [08:30<03:37, 108.91s/it]

Generated IQ data


Simulating for distance:  86%|████████▌ | 6/7 [10:40<01:56, 116.06s/it]

Generated IQ data


Simulating for distance: 100%|██████████| 7/7 [12:59<00:00, 111.42s/it]
